In [3]:
import numpy as np
import pandas as pd
import itertools
import folium
import time
from openrouteservice import *
import sys
import urllib.parse

In [1]:
ors_key = "5b3ce3597851110001cf624804ffaeec7cd246038d01eb4d3a32f633"
ors_boot = Client(ors_key)

for i in range():
    pos_1 = "35.5212561" , "139.4921609"
    pos_2 = "35.196344" , "139.0247288"
    p1r = tuple(reversed(pos_1))
    p2r = tuple(reversed(pos_2))
    
    route_rslt = ors_boot.directions((p1r,p2r),profile = "driving-car")
    tfr_time = route_rslt["routes"][0]["summary"]["duration"]